In [ ]:
import pandas as pd
import numpy as np

train_X = pd.read_csv("./data/train_features.csv").to_numpy()[:, 2:]
train_Y = pd.read_csv("./data/train_labels.csv").to_numpy()[:, 1:]

test_X = pd.read_csv("./data/test_features.csv").to_numpy()[:, 2:]
test_Y = pd.read_csv("./data/sample.csv")

In [ ]:
def extract_features(X):
    features = []
    for i in range(X.shape[0]//12):
        patient = X[i*12:i*12+12, :]
        x = []
        x = np.concatenate((x, np.nanmedian(patient, axis=0)))
        x = np.concatenate((x, np.nanmean(patient, axis=0)))
        x = np.concatenate((x, np.count_nonzero(np.isnan(patient), axis = 0)))
        features.append(x)
    return np.array(features)

train_X = extract_features(train_X) 
test_X = extract_features(test_X) 

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score
from collections import Counter


for i in range(11):
    y = train_Y[:, i]
    count = Counter(y)
    imbalance = count[0] / count[1]
    clf = xgb.XGBClassifier(scale_pos_weight=imbalance)
    model = clf.fit(train_X, y)
    pred = model.predict_proba(test_X)[:, 1]
    test_Y[test_Y.columns[i+1]] = pred
    
for i in range(11, 15):
    y = train_Y[:, i]
    clf = xgb.XGBRegressor()
    model = clf.fit(train_X, y)
    pred = model.predict(test_X)
    test_Y[test_Y.columns[i+1]] = pred
    

In [ ]:
test_Y.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')